<a href="https://colab.research.google.com/github/ffourwheel/TestSQL-PowerBi/blob/main/CreateAndInsert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 59.3 MB/s eta 0:00:00


In [ ]:
db_user = "aie222"
db_password = "112233"
db_server = "clickrouting.com"
db_name = "db_bu_student_aie222"

import mysql.connector
conn = mysql.connector.connect(host=db_server, user=db_user, password=db_password, db=db_name)
cur = conn.cursor()

# CREATE TABLE

In [ ]:
cur.execute("""
CREATE TABLE 1660903871_game_data (
    game_id INT PRIMARY KEY,
    name VARCHAR(200),
    platform VARCHAR(50),
    year INT,
    genre VARCHAR(50),
    publisher VARCHAR(100)
);
""")

In [ ]:
cur.execute("""
CREATE TABLE 1660903871_sales_data (
    sale_id INT PRIMARY KEY,
    game_id INT,
    region VARCHAR(20),
    units_sold DECIMAL(6,2),
    sale_year INT,
    price_usd DECIMAL(6,2)
);
""")

# INSERT INFO

In [ ]:
from google.colab import files
uploaded = files.upload()

import csv
with open("game_data_clean.csv", newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)

    for row in reader:
        row = [None if val == '' else val for val in row]
        cur.execute("""
            INSERT INTO 1660903871_game_data (game_id, name, platform, year, genre, publisher)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, row)

conn.commit()

Saving game_data_clean.csv to game_data_clean (6).csv


In [ ]:
from google.colab import files
uploaded = files.upload()

import csv
with open("sales_data_clean.csv", newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    header = next(reader)

    for row in reader:
        row = [None if val == '' else val for val in row]
        cur.execute("""
            INSERT INTO 1660903871_sales_data (sale_id, game_id, region, units_sold, sale_year, price_usd)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, row)

conn.commit()

Saving sales_data_clean.csv to sales_data_clean (4).csv


In [ ]:
sql = '''
SELECT * FROM 1660903871_sales_data;
'''
import pandas as pd
cur.execute(sql)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df = pd.DataFrame(rows, columns=column_names)
print(df)

     sale_id  game_id region units_sold  sale_year price_usd
0          1      156     JP       4.54       2004     70.38
1          2       35     EU       0.78       2011     22.55
2          3      189     EU       2.71       2024     66.00
3          4       93     NA       2.11       2010     48.69
4          5      182     EU       1.80       2024     37.76
..       ...      ...    ...        ...        ...       ...
589      596      130  Other       1.45       2018     38.48
590      597       60  Other       3.56       2020     32.98
591      598      151     JP       0.41       2000     41.78
592      599       31  Other       4.21       2004     63.37
593      600        5     EU       0.12       2015     18.82

[594 rows x 6 columns]


In [ ]:
## ยอดขายรวมตาม Platform
sql = """
SELECT g.platform, SUM(s.units_sold) AS total_sales
FROM 1660903871_sales_data AS s
JOIN 1660903871_game_data AS g
ON s.game_id = g.game_id
GROUP BY g.platform
ORDER BY total_sales DESC;
"""

import pandas as pd
cur.execute(sql)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df_games = pd.DataFrame(rows, columns=column_names)
df_games['total_sales'] = df_games['total_sales'].astype(int)
print('ยอดขายรวมตาม Platform')
print(df_games)

ยอดขายรวมตาม Platform
  platform  total_sales
0      PS4          344
1      3DS          324
2     X360          267
3   Switch          260
4       PC          233


In [ ]:
## ราคาเฉลี่ยต่อประเภทเกม
sql = """
SELECT g.genre, ROUND(AVG(s.price_usd), 2) AS avg_price_usd
FROM 1660903871_sales_data AS s
JOIN 1660903871_game_data AS g
ON s.game_id = g.game_id
GROUP BY g.genre
ORDER BY avg_price_usd DESC;
"""
cur.execute(sql)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df = pd.DataFrame(rows, columns=column_names)
print('ราคาเฉลี่ยต่อประเภทเกม')
print(df)

ราคาเฉลี่ยต่อประเภทเกม
     genre avg_price_usd
0   Sports         46.57
1      RPG         44.93
2   Racing         44.31
3  Shooter         43.47
4   Action         42.62


In [ ]:
## ยอดขายตามปีของแต่ละ Platform
#PS4
sql = """
SELECT
    g.platform,
    s.sale_year AS year,
    ROUND(SUM(s.units_sold), 2) AS total_units_sold,
    ROUND(SUM(s.units_sold * s.price_usd), 2) AS total_revenue_usd
FROM 1660903871_sales_data s
JOIN 1660903871_game_data g ON s.game_id = g.game_id
WHERE g.platform IN ('PS4')
GROUP BY g.platform, s.sale_year
ORDER BY s.sale_year, g.platform;
"""

import pandas as pd
cur.execute(sql)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df_platforms = pd.DataFrame(rows, columns=column_names)
print('ยอดขายแต่ละปีของ PS4')
print(df_platforms)

ยอดขายตามปีของแต่ละ Platform
   platform  year total_units_sold total_revenue_usd
0       PS4  2000            10.66            416.09
1       PS4  2001             2.53            182.84
2       PS4  2002            10.72            415.08
3       PS4  2003            19.55           1073.99
4       PS4  2004            13.33            792.85
5       PS4  2005            18.10            677.94
6       PS4  2006             8.55            526.38
7       PS4  2007             3.59            115.80
8       PS4  2008            12.07            739.51
9       PS4  2009            17.79            759.18
10      PS4  2010            26.91           1322.82
11      PS4  2011             8.83            427.00
12      PS4  2012            11.56            666.84
13      PS4  2013             6.38            253.12
14      PS4  2014            20.68            854.00
15      PS4  2015            16.63            722.86
16      PS4  2016            24.87           1277.23
17      PS4  2017

In [ ]:
#3DS
sql = """
SELECT
    g.platform,
    s.sale_year AS year,
    ROUND(SUM(s.units_sold), 2) AS total_units_sold,
    ROUND(SUM(s.units_sold * s.price_usd), 2) AS total_revenue_usd
FROM 1660903871_sales_data s
JOIN 1660903871_game_data g ON s.game_id = g.game_id
WHERE g.platform IN ('3DS')
GROUP BY g.platform, s.sale_year
ORDER BY s.sale_year, g.platform;
"""

import pandas as pd
cur.execute(sql)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df_platforms = pd.DataFrame(rows, columns=column_names)
print('ยอดขายแต่ละปีของ 3DS')
print(df_platforms)

ยอดขายแต่ละปีของ 3DS
   platform  year total_units_sold total_revenue_usd
0       3DS  2000            10.11            349.45
1       3DS  2001            22.49           1093.79
2       3DS  2002             8.51            279.21
3       3DS  2003            25.71           1102.85
4       3DS  2004            15.64            757.34
5       3DS  2005             8.70            528.25
6       3DS  2006             9.11            381.62
7       3DS  2007            11.13            391.53
8       3DS  2008            19.22            597.91
9       3DS  2009             4.12            147.60
10      3DS  2010            12.91            573.43
11      3DS  2011            14.01            586.03
12      3DS  2012            13.84            315.42
13      3DS  2013             9.47            503.76
14      3DS  2014            16.73            790.43
15      3DS  2015            13.20            485.38
16      3DS  2016            12.20            285.07
17      3DS  2017        

In [ ]:
#Switch
sql = """
SELECT
    g.platform,
    s.sale_year AS year,
    ROUND(SUM(s.units_sold), 2) AS total_units_sold,
    ROUND(SUM(s.units_sold * s.price_usd), 2) AS total_revenue_usd
FROM 1660903871_sales_data s
JOIN 1660903871_game_data g ON s.game_id = g.game_id
WHERE g.platform IN ('Switch')
GROUP BY g.platform, s.sale_year
ORDER BY s.sale_year, g.platform;
"""

import pandas as pd
cur.execute(sql)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df_platforms = pd.DataFrame(rows, columns=column_names)
print('ยอดขายแต่ละปีของ Switch')
print(df_platforms)

ยอดขายแต่ละปีของ Switch
   platform  year total_units_sold total_revenue_usd
0    Switch  2000            14.70            676.74
1    Switch  2001            14.18            653.33
2    Switch  2002             3.42            156.47
3    Switch  2003            20.82           1158.62
4    Switch  2004            11.64            532.50
5    Switch  2005            19.50            804.26
6    Switch  2006             3.51            114.31
7    Switch  2007             4.69            225.68
8    Switch  2008             6.00            244.03
9    Switch  2009             9.33            498.76
10   Switch  2010            15.70            655.43
11   Switch  2011             6.14            325.83
12   Switch  2012            15.13            787.30
13   Switch  2013             1.18             58.41
14   Switch  2014             6.26            292.51
15   Switch  2015             7.44            393.52
16   Switch  2016            16.82            868.47
17   Switch  2017     

In [ ]:
#X360
sql = """
SELECT
    g.platform,
    s.sale_year AS year,
    ROUND(SUM(s.units_sold), 2) AS total_units_sold,
    ROUND(SUM(s.units_sold * s.price_usd), 2) AS total_revenue_usd
FROM 1660903871_sales_data s
JOIN 1660903871_game_data g ON s.game_id = g.game_id
WHERE g.platform IN ('X360')
GROUP BY g.platform, s.sale_year
ORDER BY s.sale_year, g.platform;
"""

import pandas as pd
cur.execute(sql)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df_platforms = pd.DataFrame(rows, columns=column_names)
print('ยอดขายแต่ละปีของ X360')
print(df_platforms)

ยอดขายแต่ละปีของ X360
   platform  year total_units_sold total_revenue_usd
0      X360  2000            11.14            301.24
1      X360  2001             6.13            297.63
2      X360  2002            10.55            627.96
3      X360  2003             2.84            122.60
4      X360  2004            19.49           1010.38
5      X360  2005            12.46            608.63
6      X360  2006             2.34             24.78
7      X360  2007            16.97            906.27
8      X360  2008            19.59            966.17
9      X360  2009             6.93            282.01
10     X360  2010            11.35            391.85
11     X360  2011             5.68            185.96
12     X360  2012            16.56            605.36
13     X360  2013             7.25            123.35
14     X360  2014            16.55            949.44
15     X360  2015             7.93            617.12
16     X360  2016             5.11            276.73
17     X360  2017       

In [ ]:
#PC
sql = """
SELECT
    g.platform,
    s.sale_year AS year,
    ROUND(SUM(s.units_sold), 2) AS total_units_sold,
    ROUND(SUM(s.units_sold * s.price_usd), 2) AS total_revenue_usd
FROM 1660903871_sales_data s
JOIN 1660903871_game_data g ON s.game_id = g.game_id
WHERE g.platform IN ('PC')
GROUP BY g.platform, s.sale_year
ORDER BY s.sale_year, g.platform;
"""

import pandas as pd
cur.execute(sql)
rows = cur.fetchall()
column_names = [i[0] for i in cur.description]
df_platforms = pd.DataFrame(rows, columns=column_names)
print('ยอดขายแต่ละปีของ PC')
print(df_platforms)

ยอดขายแต่ละปีของ PC
   platform  year total_units_sold total_revenue_usd
0        PC  2000             6.98            400.15
1        PC  2001            17.77            722.16
2        PC  2002             9.37            418.11
3        PC  2003             9.40            573.45
4        PC  2004            11.43            795.93
5        PC  2005             1.21             33.14
6        PC  2006             7.15            379.45
7        PC  2007             9.04            409.29
8        PC  2008            13.81            510.66
9        PC  2009             3.02            192.96
10       PC  2010             3.01             67.88
11       PC  2011            22.50           1034.96
12       PC  2012             8.73            525.75
13       PC  2013             3.62            187.96
14       PC  2014            11.68            493.04
15       PC  2015             5.26             59.73
16       PC  2016            10.44            494.06
17       PC  2017         